# Report for the project "Identify Fraud from Enron Mail"

## Table of Contents
<ul>
<li><a href="#intro">Introduction</a></li>
<li><a href="#data_exploration">Data exploration</a></li>
<li><a href="#features">Feature selection and engineering</a></li>
<li><a href="#algorithm">Algorithm choice and tuning</a></li>
<li><a href="#validation">Validation and evaluation</a></li>
<li><a href="#conclusion">Conclusions</a></li>
<li><a href="#ref">References</a></li>
</ul>

<a id='intro'></a>
## Introduction

In this project, the aim is to create an algorithm that can **identify the Persons of Interest (POIs)** in the fraud case known as the **Enron scandal** (<a href="#ref 1">ref 1</a>). As described in the introduction of the project, the POIs are defined as "*individuals who were indicted, reached a settlement or plea deal with the government, or testified in exchange for prosecution immunity*". 

We are in the frame of labeled data (POI = True or POI = False), so the **POI identifier** will be of the family of **supervised Machine Learning**. Besides, the output is discrete, so we are dealing with a **classifier** like naive Bayes, Support Vector Machine (SVM), or Decision Tree (to mention only a few of them).   

In the next sections, we will start by having a general look at the dataset (which information is available and could be used by the classifier?). We will then select, create and transform features so that to be able to feed the classifier with the information we think are appropriate for the task we want it to perform. Using this reduced, customized set of features, we will then try different kinds of classifiers, and perform a systematic "grid search" so that to best tune its parameters. The performance of the classifier will be evaluated using a specific validation scheme and dedicated metrics.  

The Python code used for the project is attached as POI_id.py file and also in Jupyter notebook (.ipynb and exported as pdf).

<a id='data_exploration'></a>
## Data exploration

### Data overview
The initial dataset, loaded from final_project_dataset.pkl, is a dictionary of dictionaries. The keys of the "mother" dictionary are the persons for which we have email and financial data, like 'METTS MARK', 'BAXTER JOHN C' or 'ELLIOTT STEVEN'.  

There are **146 people (i.e. samples) in the dataset**. Let's have a look at 1 sample, for instance 'METTS MARK':

```python
{'bonus': 600000,
 'deferral_payments': 'NaN',
 'deferred_income': 'NaN',
 'director_fees': 'NaN',
 'email_address': 'mark.metts@enron.com',
 'exercised_stock_options': 'NaN',
 'expenses': 94299,
 'from_messages': 29,
 'from_poi_to_this_person': 38,
 'from_this_person_to_poi': 1,
 'loan_advances': 'NaN',
 'long_term_incentive': 'NaN',
 'other': 1740,
 'poi': False,
 'restricted_stock': 585062,
 'restricted_stock_deferred': 'NaN',
 'salary': 365788,
 'shared_receipt_with_poi': 702,
 'to_messages': 807,
 'total_payments': 1061827,
 'total_stock_value': 585062}
```

We can check that, as stated in the project details page, each sample has:
- 1 label (poi)
- 14 financial features
- 6 email features

Looping through the dictionaries, it is found that there is **no NaN for the 'poi' field**. **Among the 146 samples, there are 18 POIs and 128 non POIs**. The good news is that there is no missing label in this already quite small dataset. Bad news is that **the two classes are strongly imbalanced** (12% POIs, 88% non POIs). This will make us use a stratified shuffling method for the validation of the proposed classifier (see section <a href="#validation">Validation and evaluation</a>).  

### Further exploration of the available features

#### Financial features

In order to have a further look at the available features, a short function was written to get, for a given numerical feature, the count of NaN, the min, the max, the average values for the POI and non POI samples. The boxplot and histogram are also plotted. The aim of this function is to ease the **identification of missing values and outliers**, and also to **get a first hint on the features that might be relevant for our classifier**.

When used for the first time for the visualization of the 'salary' feature, the boxplot and histogram were completely "shrinked" due to the presence of an extreme outlier (salary above 25 M$). A second function was written so that to display the samples for which an input feature is above a given threshold. Using this function, we found that this **outlier, with key 'TOTAL'**, do not correspond to a person but contains the **sum of each financial feature for all the other samples**: 

```python
TOTAL
{'bonus': 97343619,
 'deferral_payments': 32083396,
 'deferred_income': -27992891,
 'director_fees': 1398517,
 'email_address': 'NaN',
 'exercised_stock_options': 311764000,
 'expenses': 5235198,
 'from_messages': 'NaN',
 'from_poi_to_this_person': 'NaN',
 'from_this_person_to_poi': 'NaN',
 'loan_advances': 83925000,
 'long_term_incentive': 48521928,
 'other': 42667589,
 'poi': False,
 'restricted_stock': 130322299,
 'restricted_stock_deferred': -7576788,
 'salary': 26704229,
 'shared_receipt_with_poi': 'NaN',
 'to_messages': 'NaN',
 'total_payments': 309886585,
 'total_stock_value': 434509511}
```

This **outlier** was **removed from the dataset** as it will not help for the discrimination between POIS and non-POIs. 

When used on the updated dataset, the feature exploration function returns the following output for the 'salary' feature: 

In [2]:
# Image display (the graphs were generated by the POI_id code, but are displayed in this report as pictures)
from IPython.display import Image

![feature explore salary](feature_explore_salary.png)

This visualization highlights three interesting points:
- one third of the salary data is missing, but at least we have it for all the POI except 1
- the median salary of the POI sugroup is higher than that of the non POI subgroup. Interesting feature for our classifier!
- **3 clear outliers**: 2 POI and 1 non POI
- the 'shape' of the salary distribution seems to be quite similar for POI and non POI 

The "outlier display" function (using a threshold of 1M$ on salary) was used to identify the 3 clear outliers mentioned above:
- the 2 POIs are **Kenneth Lay** and **Jeffrey Skilling**, respectively Chairman of the Board of Directors and COO of Enron when the scandal occured.
- the non-POI is **Mark Frevert**, Enron vice chairman.

We want to **keep these outliers** in the dataset, for several reasons:
- contextual reason: we are dealing with a fraud, so **outliers for any kind of financial feature might be very relevant information** that we want to use in our algorithm
- practical reason: the **dataset is already quite small** (less than 150 samples), and the number of samples of the POI class is even more reduced (18 samples). We cannot really afford throwing samples if it is not mandatory for the good functioning of the classifier.

The same feature visualization process was systematically performed for all the other financial features. It turns out that the financial features can be splitted into **four categories**:
- the **overall features**, 'total_payments' and 'total_stock_value', which, by summing the different financial features, have a value for a maximum number of people.
- the **features with too many missing values** (typically more than 50%): 'loan_advances', 'director_fees', 'deferral_payments', 'deferred_income', 'restricted_stock_deferred'.
- the **features for which little difference is observed between POIs and non-POIs**: 'other', 'expenses'.
- the **remaining features**, that is the features for which data are available for a significant portion of the samples, and whose visualization higlights valuable difference between POIs and non-POIs: 'bonus', 'exercised_stock_options', 'long_term_incentive', 'restricted_stock', 'salary'. 

Based on this finding, **2 strategies** were tested for the financial features:
- **strategy A**: we keep only the 2 overall features: 'total_payments' and 'total_stock_value'
- **strategy B**: we keep only the 5 features mentioned above: 'bonus', 'exercised_stock_options', 'long_term_incentive', 'restricted_stock', 'salary'. 
Strategy A has the advantage of using a very limited of features that show very little missing data. On the other hand, summing all financial data may result in 'diluting' the information we need to distinguish between POIs and non POIs, hence strategy B. 

In both strategy, as said before **we decide to keep the outliers** of the selected features. Indeed, we saw that there were at least two very emblematic POIs, Kenneth Lay and Jeffrey Skilling, who presented outliers for one or several financial features. Given the limited number of POIs in the dataset, we don't want to further reduce their proportion in the data that will be used to train the algorithm. 


#### Email features

The feature explore function was also used to visualize the numerical email features (i.e all email features except from the email address). As an example, the following is observed for the feature 'shared_receipt_with_poi':

![feature explore shared receipt POI](feature_explore_shared_receipt_POI.png)

At first sight, the email feature **'shared_receipt_with_poi'** looks **promising for our classification**: data is available for a large portion of the samples and there is an interesting difference of central tendency between the POIs and non-POIs.  

On the other hand, the **'from_messages' and 'to_messages'** features do not seem to carry valuable information here. Nevertheless, they **can be used to normalize the features 'from_this_person_to_poi' and 'from_poi_to_this_person'**, which, as is, are not so expressive.  Indeed, someone that tends to send many emails is more likely to also send more emails to POIs than another person. 

If we make the **assumption that the fraud system was established by a limited number of people who know each others**, then it is not the absolute number of mails send to or received from a POI that matters, but the ration to the total mailbox. If A and B send 30% and 5% of their emails to C respectively, then C is likely to be closer to A than B. Later on, we will check this intuition by using the original poi email features, then replacing them by the **'relative' poi email features**, and comparing the erformance of teh POI identifier algorithm.

Finally, in the frame of our POI identifier algorithm, there is no need to use the 'email_address' feature, which was therefore disregarded. 

<a id='features'></a>
## Feature selection and engineering

### "Relative POI email features" creation
As explained above, the piece of code given below was used to implement 2 new emails features (when input email data were available) :
- 'relative_from_poi_to_this_person': the ratio of emails sent to a POI over total number of sent emails
- 'relative_from_this_person_to_poi': the ratio of emails received from a POI over the total number of received emails feature 
```python
for k in my_dataset.keys():
    if my_dataset[k]['from_poi_to_this_person'] != 'NaN' and my_dataset[k]['to_messages'] != 'NaN':
        my_dataset[k]['relative_from_poi_to_this_person'] = \
        float(my_dataset[k]['from_poi_to_this_person'])/float(my_dataset[k]['to_messages'])
    else: 
        my_dataset[k]['relative_from_poi_to_this_person'] = 'NaN'
        
    if my_dataset[k]['from_this_person_to_poi'] != 'NaN' and my_dataset[k]['from_messages'] != 'NaN':
        my_dataset[k]['relative_from_this_person_to_poi'] = \
        float(my_dataset[k]['from_this_person_to_poi'])/float(my_dataset[k]['from_messages'])
    else:
        my_dataset[k]['relative_from_this_person_to_poi'] = 'NaN'
```

These 2 new features were visualized as a scatterplot:

![scatter relative from to features](scatter_relative_from_to.png)

Interestingly **all the POIs** have a ratio of emails sent to other POI equal or above about 0.2, that is they all **sent 1 email in 5 to another POI**. Although seemingly less clear, the 'relative_from_poi_to_this_person' feature is above 0.02 for all POIs. It was decided to include these 2 features in the feature list for the classifier.

### Rescaling
In the investigation, we want to try different kinds of algorithms for our classifier. Some of them, like SVM, would behave differently whether applied to scaled or unscaled features. In our case, we decided, as explained above, not to remove the outliers in the financial features. Because of this choice, min/max scaling would not be a good approach for scaling the data. 

We decide to **start by using the unscaled features**. If we cannot manage to get the desired performance for our classifier (better than 0.3 in both recall and precision), we will go back to this step and see which scaling method could be used.

### Feature selection
Based on the findings and discussions above, 4 sets of features were finally compared: 
- the **list 3** is our **best manual selection**, including: 
    - the 5 'most promising' financial features (strategy A): 'bonus', 'exercised_stock_options', 'long_term_incentive', 'restricted_stock', 'salary'
    - the 'shared_receipt_with_poi'and the two 'relative' POI email features  
- the lists 1 and 3 differ by the used financial features (2 'total' features vs. 5 'handpicked' features)
- the lists 2 and 3 differ by the used email features ('absolute' vs. 'relative' POI mail features)
- the list 4 is a list of the 5 features (among the original features), selected using SelectKBest class. The idea is to see what would be the performance of the classifier if almost no manual exploration of the features had been performed.  

In practice, the following piece of code was used to define list 4:

```python
# List 4: 'best' 5 features, in the sense of higher f-score
# Select all features except email_address
features_list4 = ['poi','bonus','deferral_payments','director_fees','exercised_stock_options','expenses',\
                  'from_messages','from_poi_to_this_person','from_this_person_to_poi','loan_advances',\
                  'long_term_incentive','other','restricted_stock','restricted_stock_deferred','salary',\
                  'shared_receipt_with_poi','to_messages','total_payments','total_stock_value'] 
# Extract features and labels from dataset
data4 = featureFormat(my_dataset, features_list4, sort_keys = True)
labels4, features4 = targetFeatureSplit(data4)
# Use SelectKBest to select the features having the 5 highest f-score
from sklearn.feature_selection import SelectKBest
Kbest_selector = SelectKBest(k=5)
features4_new = Kbest_selector.fit_transform(features4, labels4)
```

The features index 3 ('exercised_stock_options'), 17 ('total_stock_value'), 0 ('bonus'), 13 ('salary') and 9 ('long_term_incentive') were selected, with f1-score from 25 to 25. Note that there is no email features in this selection.

### Feature transformation
These 4 lists do not contain a very high number of features, and therefore we do **no**t see the **need, as a first approach, to use a dimensionality reduction technique like PCA**. If the performance of the classifier is below expectation we will go back to this step and try to use a PCA on a the list number 3.

### Feature lists benchmarking using Naive Bayes classifier 
A short function was written to compare the performance of the 4 feature lists, using the same Naive Bayes classifier, splitting the dataset in half (50% training, 50% test) in a stratified fashion, and looking at the recall and precision metrics. 

When interpreting the results, we are **mainly focusing in the recall** and also precision for the POI class. Indeed, we are in a case of very imbalanced classification, so we want to ensure that our algorithm will be able to spot the rare positive samples in the dataset (i.e "high" recall). Given the context of this analysis, we want to give the priority at the recall over the precision. Indeed, the algorithm could be used as a first filter to detect people for which further investigation may lead to findings. The precision may not be very high, but at least we will minimize the missing of true POIs.

The **best results** (precision=0.40 and recall=0.22 on test dataset for the POI class) are **obtained with the list 3**, so this list of features is **used from that point in the project**. Nevertheless, tuning of the algorithm is needed to reach the target performance. 

Using this list of **8 features**, the **dataset** contains **133 samples among which** there are **18 POIs**.
The **POI class** represents **only 13.5% of the dataset**.

<a id='algorithm'></a>
## Algorithm choice and tuning

As for the feature lists benchmarking, the dataset is first splitted (in stratified manner because we are dealing with imbalanced classes) into 50% training set, 50% test set. 

Then four types of classifiers are tested: Naive Bayes (NB), Support Vector Machine (SVM), k-nearest neighbors (kNN) and Decision Tree (DT). In the case of SVM, kNN and DT, a grid search is performed to best tune, in terms of recall score, the key parameters of these classifiers. 

The performance of the NB classifier and of the "tuned" SVM, kNN and DT classifiers is evaluated by looking at three metrics: recall score, precision score, and f1-score. 

### Naive Bayes
The following results were obtained for the Naive Bayes classifier:
```python
             precision    recall  f1-score   support

        0.0       0.89      0.95      0.92        58
        1.0       0.40      0.22      0.29         9
```
As can be seen, the recall of POIs is below target (0.3).

### Support vector Machine
The following grid search was performed:
- two different kernel types: rbf and sigmoid. Based on the visualization we did before, we think it would be difficult for a linear kernel to be able to separate well between POIs and non-POIs.
- three different values for the gamma kernel coefficient: 1, 0.1 and 0.01
- four different values for the penalty parameter C: 10, 100, 1000 and 10000


The following set of parameters gave the best recall score on training set: {'C': 10, 'gamma': 1, 'kernel': 'sigmoid'}. Note that the main tested parameter having an effect in our classification task seemed to be the kernel: sigmoid kernel worked better than rbf kernel. 

When used on the test set, the results below were obtained:
```python
             precision    recall  f1-score   support

        0.0       0.75      0.05      0.10        58
        1.0       0.13      0.89      0.22         9
```
The recall score for POI class is high, but the precision score is very low and below target.

### k-nearest neighbors
The following grid search was performed:
- five different numbers of neighbors: 2, 3, 4, 5, 6 
- three different algorithms: ‘ball_tree’, ‘kd_tree’, ‘brute’

The following set of parameters gave the best recall score on training set: {'algorithm': 'ball_tree', 'n_neighbors': 3}. Note that setting the number of neighbors to 3 significantly improved the performance of the kNN classifier, whereas the type of algorithm had no visible effect.

When used on the test set, the results below were obtained:
```python
             precision    recall  f1-score   support

        0.0       0.90      0.93      0.92        58
        1.0       0.43      0.33      0.38         9
```
The kNN classifier works with acceptable recall and precision when tuned with a number of neighbors of 3.

### Decision Tree
The following grid search was performed:
- two kinds of criterions: 'gini' and 'entropy'
- four different max_depth parameters: 1, 2, 4 and 8
- three different min_samples_split settings: 2, 4 and 8

The following set of parameters gave the best recall score on training set: {'criterion': 'gini', 'max_depth': 1, 'min_samples_split': 2}. The best performance was achieved with a maximum_depth parameter of 1. With such value for maximum_depth, the type of criterion (gini or entropy) and the min_samples_split parameters seem to have little effect. 

When used on the test set, the results below were obtained:
```python
             precision    recall  f1-score   support

        0.0       0.94      0.81      0.87        58
        1.0       0.35      0.67      0.46         9
```
A decision tree classifier with a maximum_depth parameter of 1 gives the **best trade-off with a recall score of 0.67 for the POI class and a precision of 0.35**, also above minimum target. We propose to submit this algorithm (**Decision Tree classifier, {'criterion': 'gini', 'max_depth': 1, 'min_samples_split': 2}**) for test using the tester code.

<a id='validation'></a>
## Validation and evaluation

The **validation** is **done re-using** the provided **tester.py** code. Note that the code has been slightly modified so that to work with model_selection.StratifiedShuffleSplit instead of the previous version cross_validation.StratifiedShuffleSplit. 

In order to get a **robust estimate of the classifier performance**, and knowing that the dataset is small and imbalanced, the strategy adopted in tester.py is to automatically (and randomly) define 1000 folds of the dataset where the percentage of samples of POI and non-POI class is preserved. As explained in the documentation page (<a href="#ref 2">ref 2</a>), "***stratified random splits do not guarantee that all folds will be different***", especially here where the dataset has a small size. 

Each fold is splitted into 50% training dataset and 50% test dataset, still preserving the percentage of POI and non-POI samples. For each fold, the total number of true negatives (TN), false negatives (FN), false positives (FP) and true positives (TP) is updated.

At the end of the loop over the 1000 folds, the accuracy, precision, recall, f1 and f2 scores are calculated for the overall number of predictions. These metrics are used for the evaluation of the classifier.  The following results are obtained:
- Accuracy: 0.76071	
- **Precision: 0.32989**	
- **Recall: 0.65450**	
- F1: 0.43867	
- F2: 0.54688
- Total predictions: 14000	
- True positives: 1309	
- False positives: 2659	
- False negatives:  691	
- True negatives: 9341

**This exhaustive validation confirms that with the chosen feature list and tuned Decision Tree classifier both the recall and precision are above 0.30.** The recall is higher than the precision, which is also in line with the trade-off we were ready to make given the context of the analysis: in the frame of a fraud investigation we prefer further investigating what may turn out to be false positives, than missing a false negatives.  

<a id='conclusion'></a>
## Conclusions

After exploring the initial dataset and removing an obvious outlier, 4 different sets of features were benchmarked. The most promising **feature list** was the one **composed of the 5 "handpicked" financial features, 1 existing email feature (shared receipt with POI) and 2 created email features (relative "to POI" and "from POI" number of messages)**. 

This feature list was used on 4 different types of classifiers, 3 of which being subject to a grid search for best tuning of their main parameters. At that stage, a "light" validation was performed on a 50-50 stratified split of the dataset, and the evaluation was primarly based on recall score (although precision score was also monitored as priority 2 metrics). The **decision tree classifier, with a maximum depth of 1, a minimum samples split of 2 and using the 'gini' criterion'** was found to give the best results. 

The performance of this classifier using this feature list was thoroughly **validated on a 1000 folds stratified shuffle split of the dataset**, giving a **recall score of 0.65 and a precision score of 0.33**. 

Although the proposed algorithm performs above minimum target (0.30), a higher performance POI identifier could be designed by using methods and applying tips that are especially efficient when dealing with imbalanced binary dataset: oversampling of the minority class, use anomaly detection methods... (see <a href="#ref 3">ref 3</a>, <a href="#ref 4">ref 4</a> and <a href="#ref 5">ref 5</a>).

<a id='ref'></a>
## References

<a id='ref 1'></a> ref 1: https://en.wikipedia.org/wiki/Enron_scandal

<a id='ref 2'></a> ref 2: https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.StratifiedShuffleSplit.html

<a id='ref 3'></a> ref 3: https://stats.stackexchange.com/questions/235808/binary-classification-with-strongly-unbalanced-classes

<a id='ref 4'></a> ref 4: http://www.svds.com/learning-imbalanced-classes/

<a id='ref 5'></a> ref 5: http://www.ele.uri.edu/faculty/he/PDFfiles/ImbalancedLearning.pdf